In [1]:
# Import dependencies
import os
os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import pandas as pd
import numpy as np
import tensorflow as tf
import logging
from tqdm import tqdm

# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, BinaryFocalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, NUM_CLASSES, TF_SHUFFLE_SIZE, TF_BATCH_SIZE
from src.model.tensorflow_utils import load_dataset, apply_augmentation_to_dataset, oversample_minority_classes, optimize_dataset, count_dataset_size
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_loss_monitor, setup_garbage_collector, get_metrics
from src.model.tensorflow_utils import calculate_class_weights, show_class_weights, start_or_resume_training, analyze_class_distribution
from src.model.densnet.tensorflow_dense_net_basic import build_densenet

# Input Data
initial_epoch   = 27
resume_training = True
checkpoint_path = '/Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v5/checkpoints/cp-0026.keras'
model_name      = "Simplified_DenseNet_v5"

In [2]:
train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)
test_ds  = load_dataset(f"{TF_RECORD_DATASET}/test.tfrecord", TF_BUFFER_SIZE)

I0000 00:00:1742457596.730415 20321893 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1742457596.730715 20321893 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Optimize Dataset for rare clasess
class_weights = calculate_class_weights(train_ds, NUM_CLASSES)
train_ds      = oversample_minority_classes(train_ds, class_weights)
class_weights = calculate_class_weights(train_ds, NUM_CLASSES)

In [4]:
steps_per_epoch  = int(count_dataset_size(train_ds, None) / TF_BATCH_SIZE)
validation_steps = int(count_dataset_size(val_ds, None) / TF_BATCH_SIZE)

# Testing
train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = apply_augmentation_to_dataset(train_ds)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)

val_ds  = optimize_dataset(val_ds, TF_BATCH_SIZE)
test_ds = optimize_dataset(test_ds, TF_BATCH_SIZE)

Counting samples: 108109 samples [01:10, 1541.59 samples/s] 
Counting samples: 15391 samples [00:18, 852.88 samples/s]


In [5]:
# Setup Model Deps
# Setup Loggers
logger            = setup_logger()
training_logger   = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor   = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
loss_monitor      = setup_loss_monitor(MODELS_PATH, model_name, logger, val_ds, resume_training=resume_training, initial_epoch=initial_epoch)
garbage_collector = setup_garbage_collector(logger)
metrics           = get_metrics()

# Setup compile arguments
focal_loss = BinaryCrossentropy(from_logits=False, label_smoothing=0.01)
reduce_lr  = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)

epoch_mode           = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint           = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max")

model_path      = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max")

2025-03-20 09:03:54 - INFO - Resuming from existing metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v5/train_metrics.csv
2025-03-20 09:03:54 - INFO - Resuming from existing validation metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v5/val_metrics.csv
2025-03-20 09:03:54 - INFO - Cleaned training metrics file, kept 87828 records before epoch 27
2025-03-20 09:03:54 - INFO - Cleaned validation metrics file, kept 26 records before epoch 27
2025-03-20 09:03:54 - INFO - Found 87828 existing training records
2025-03-20 09:03:54 - INFO - Found 26 existing validation records
2025-03-20 09:03:54 - INFO - Cleaned loss analysis metrics file, kept 390 records before epoch 27
2025-03-20 09:03:54 - INFO - Resuming from existing loss analysis file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v5/loss_analysis_metrics.csv


In [6]:
# Model Training
model          = build_densenet(NUM_CLASSES)
compile_kwargs = {'optimizer': Adam(learning_rate=1e-4, clipnorm=1.0), 'loss': focal_loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    30,
    steps_per_epoch, 
    validation_steps, 
    class_weights=class_weights,
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, loss_monitor, garbage_collector], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-03-20 09:03:55 - INFO - Resuming from existing metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v5/train_metrics.csv
2025-03-20 09:03:55 - INFO - Resuming from existing validation metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v5/val_metrics.csv
2025-03-20 09:03:55 - INFO - Cleaned training metrics file, kept 87828 records before epoch 27
2025-03-20 09:03:55 - INFO - Cleaned validation metrics file, kept 26 records before epoch 27
2025-03-20 09:03:55 - INFO - Found 87828 existing training records
2025-03-20 09:03:55 - INFO - Found 26 existing validation records
2025-03-20 09:03:55 - INFO - Cleaned loss analysis metrics file, kept 390 records before epoch 27
2025-03-20 09:03:55 - INFO - Resuming from existing loss analysis file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v5/loss_analysis_metrics.csv


Loading full model from checkpoint: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v5/checkpoints/cp-0026.keras


2025-03-20 09:03:56 - INFO - 
=== Training Started ===

2025-03-20 09:03:56 - INFO - Batch Size: 32
2025-03-20 09:03:56 - INFO - Optimizer: Adam
2025-03-20 09:03:56 - INFO - 

2025-03-20 09:03:56 - INFO - 
=== Starting Epoch 27 ===



Epoch 27/30
3378/3378 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step - accuracy: 0.9076 - auc: 0.7619 - f1_score: 0.1050 - loss: 0.1843 - precision: 0.4878 - recall: 0.0750   

2025-03-20 09:29:45 - INFO - 
=== Epoch 27 Summary ===
2025-03-20 09:29:45 - INFO - Time: 1548.99s
2025-03-20 09:29:45 - INFO - Training   - accuracy: 0.9078 - auc: 0.7653 - f1_score: 0.1072 - loss: 0.1834 - precision: 0.4877 - recall: 0.0764 - learning_rate: 0.0001
2025-03-20 09:29:45 - INFO - Validation - accuracy: 0.8988 - auc: 0.7139 - f1_score: 0.1041 - loss: 0.3108 - precision: 0.2660 - recall: 0.1043
2025-03-20 09:29:45 - INFO - ==================================================



...

2025-03-20 11:45:58 - INFO - 
=== Epoch 30 Summary ===
2025-03-20 11:45:58 - INFO - Time: 3928.09s
2025-03-20 11:45:58 - INFO - Training   - accuracy: 0.9079 - auc: 0.7717 - f1_score: 0.1189 - loss: 0.1804 - precision: 0.4923 - recall: 0.0865 - learning_rate: 0.0001
2025-03-20 11:45:58 - INFO - Validation - accuracy: 0.9091 - auc: 0.7389 - f1_score: 0.0735 - loss: 0.2717 - precision: 0.3159 - recall: 0.0532
2025-03-20 11:45:58 - INFO - ==================================================



1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

2025-03-20 11:47:22 - INFO - 
Loss Analysis - Atelectasis
2025-03-20 11:47:22 - INFO - Confidence Distribution:
2025-03-20 11:47:22 - INFO - -- High (>0.9): 0.00%
2025-03-20 11:47:22 - INFO - -- Medium (0.6-0.9): 0.02%
2025-03-20 11:47:22 - INFO - -- Uncertain (0.4-0.6): 0.99%
2025-03-20 11:47:22 - INFO - -- Low (<0.4): 98.99%
2025-03-20 11:47:22 - INFO - Performance:
2025-03-20 11:47:22 - INFO - -- True Positives: 10
2025-03-20 11:47:22 - INFO - -- False Positives: 17
2025-03-20 11:47:22 - INFO - -- Loss Contribution: 0.3285
2025-03-20 11:47:22 - INFO - Average Confidence:
2025-03-20 11:47:22 - INFO - -- Correct Predictions: 9.18%
2025-03-20 11:47:22 - INFO - -- Incorrect Predictions: 15.39%
2025-03-20 11:47:22 - INFO - 
Loss Analysis - Cardiomegaly
2025-03-20 11:47:22 - INFO - Confidence Distribution:
2025-03-20 11:47:22 - INFO - -- High (>0.9): 0.28%
2025-03-20 11:47:22 - INFO - -- Medium (0.6-0.9): 1.36%
2025-03-20 11:47:22 - INFO - -- Uncertain (0.4-0.6): 1.40%
2025-03-20 11:47:22

3378/3378 ━━━━━━━━━━━━━━━━━━━━ 4015s 1s/step - accuracy: 0.9078 - auc: 0.7685 - f1_score: 0.1166 - loss: 0.1812 - precision: 0.4929 - recall: 0.0851 - val_accuracy: 0.9091 - val_auc: 0.7389 - val_f1_score: 0.0735 - val_loss: 0.2717 - val_precision: 0.3159 - val_recall: 0.0532 - learning_rate: 1.0000e-04


2025-03-20 11:47:25 - INFO - 
=== Training Completed! ===

2025-03-20 11:47:25 - INFO - Final Metrics: accuracy: 0.9079 - auc: 0.7717 - f1_score: 0.1189 - loss: 0.1804 - precision: 0.4923 - recall: 0.0865 - val_accuracy: 0.9091 - val_auc: 0.7389 - val_f1_score: 0.0735 - val_loss: 0.2717 - val_precision: 0.3159 - val_recall: 0.0532



In [7]:
# Initialize the evaluator
from src.model.tensorflow_model_evaluation import ModelEvaluation

mappings_path = f"{TF_RECORD_DATASET}/label_mappings.csv"
evaluator = ModelEvaluation(
    model=model, 
    model_name=model_name, 
    test_dataset=test_ds,
    label_mappings_path=mappings_path,
    output_dir=MODELS_PATH
)

# Cell 2: Generate predictions and basic metrics
metrics = evaluator.evaluate()


Generating predictions...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1

In [8]:
evaluator.generate_classification_report()


Classification Report:
                    precision    recall  f1-score   support

       Atelectasis       0.48      0.01      0.02      1719
      Cardiomegaly       0.33      0.29      0.31       413
     Consolidation       0.00      0.00      0.00       687
             Edema       0.12      0.47      0.20       342
          Effusion       0.65      0.15      0.24      1984
         Emphysema       0.29      0.17      0.22       376
          Fibrosis       0.10      0.03      0.04       252
            Hernia       0.41      0.38      0.39        34
      Infiltration       0.49      0.06      0.10      2943
              Mass       0.50      0.00      0.01       861
        No Finding       0.90      0.01      0.03      7717
            Nodule       0.00      0.00      0.00       943
Pleural_Thickening       0.00      0.00      0.00       503
         Pneumonia       0.00      0.00      0.00       210
      Pneumothorax       0.36      0.14      0.20       789

         micro

In [9]:
evaluator.plot_confusion_matrices()


Generating combined confusion matrix...
Saved combined confusion matrix to: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v5/combined_confusion_matrix.png


In [10]:
for image, labels in test_ds.take(1):
    if len(image.shape) == 4:
        image = image[0]
        labels = labels[0]
    
    evaluator.visualize_prediction(image=image, true_labels=labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 850ms/step
